In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

In [ ]:
def dibujar_puntos(imagen, punto_referencia, punto_marcado):
    cv2.circle(imagen, punto_referencia, 5, (0, 0, 255), -1)  # Punto de referencia en rojo
    cv2.circle(imagen, punto_marcado, 5, (255, 0, 0), -1)  # Punto marcado en azul

In [ ]:
def crear_matriz(columnas, filas):
    return np.zeros((filas, columnas), dtype=int)

In [ ]:
def calcular_centro_grilla(x_ref, y_ref, pixeles_por_mm, num_filas):
    centro_x = x_ref
    centro_y = y_ref  # Ajustar para que la grilla se dibuje desde el punto de referencia hacia arriba
    tamanio_celda = pixeles_por_mm
    return centro_x, centro_y, tamanio_celda

In [ ]:
def dibujar_grilla(imagen_objetivo, centro_x, centro_y, ancho_celda, num_columnas, num_filas, matriz_valores):
    
    # Dibujar líneas horizontales
    for fila in range(num_filas + 1):
        posicion_y = int(centro_y - fila * ancho_celda)
        inicio_horizontal = int(centro_x - (num_columnas // 2) * ancho_celda)
        fin_horizontal = int(centro_x + (num_columnas // 2) * ancho_celda)
        cv2.line(imagen_objetivo, (inicio_horizontal, posicion_y), (fin_horizontal, posicion_y), (0, 255, 0), 1)
    
    # Dibujar líneas verticales
    for columna in range(num_columnas + 1):
        posicion_x = int(centro_x - (num_columnas // 2) * ancho_celda + columna * ancho_celda)
        inicio_vertical = int(centro_y - num_filas * ancho_celda)
        fin_vertical = int(centro_y)
        cv2.line(imagen_objetivo, (posicion_x, inicio_vertical), (posicion_x, fin_vertical), (0, 255, 0), 1)
    
    # Insertar valores en las celdas
    for fila in range(num_filas):
        for columna in range(num_columnas):
            valor = matriz_valores[fila][columna]
            coord_x = int(centro_x - (num_columnas // 2) * ancho_celda + columna * ancho_celda + ancho_celda // 2)
            coord_y = int(centro_y - fila * ancho_celda - ancho_celda // 2)
            cv2.putText(imagen_objetivo, str(valor), (coord_x - 10, coord_y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (250, 25, 25), 1, cv2.LINE_AA)

## Posicion en la grilla


#### Cálculo de `grilla_x` (Índice de Columna)

$$
\text{grilla\_x} = \left\lfloor \frac{\text{x\_marcado} - \text{centro\_x} + \left(\frac{\text{num\_columnas}}{2}\right) \times \text{ancho\_celda}}{\text{ancho\_celda}} \right\rfloor
$$

#### Cálculo de `grilla_y` (Índice de Fila)

$$
\text{grilla\_y} = \left\lfloor \frac{\text{centro\_y} - \text{y\_marcado}}{\text{ancho\_celda}} \right\rfloor
$$

In [ ]:
def obtener_posicion_grilla(x_marcado, y_marcado, centro_x, centro_y, tamanio_celda, num_columnas, num_filas):
    
    grilla_x = int((x_marcado - centro_x + (num_columnas // 2) * tamanio_celda) / tamanio_celda)
    grilla_y = int((centro_y - y_marcado) / tamanio_celda)
    
    return grilla_x, grilla_y

In [ ]:
# Lista para almacenar los puntos marcados
puntos_marcados = []

# Función de callback para eventos del mouse
def marcar_puntos(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        puntos_marcados.append((x, y))
        cv2.circle(param, (x, y), 5, (255, 0, 0), -1)  # Dibuja el punto en la imagen
        cv2.imshow('Imagen con Grilla', param)

# Mostrar la imagen y configurar el callback del mouse
def mostrar_imagen_con_mouse(imagen):
    cv2.imshow('Imagen con Grilla', imagen)
    cv2.setMouseCallback('Imagen con Grilla', marcar_puntos, imagen)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Función para actualizar y guardar el dataset
def actualizar_dataset(ruta_dataset, puntos_marcados, nombre_imagen, punto_referencia, mm_por_pixel):
    try:
        # Cargar el dataset existente o crear uno nuevo si no existe
        if os.path.exists(ruta_dataset):
            df = pd.read_csv(ruta_dataset)
        else:
            df = pd.DataFrame(columns=['imagen', 'punto_x', 'punto_y', 'mm_por_pixel'] +
                                       [f'x_punto {i}' for i in range(1, 16)] +
                                       [f'y_punto {i}' for i in range(1, 16)])

        # Crear una nueva fila con las coordenadas de los puntos marcados
        nueva_fila = {
            'imagen': nombre_imagen,
            'punto_x': punto_referencia[0],
            'punto_y': punto_referencia[1],
            'mm_por_pixel': mm_por_pixel,
        }

        for i, (x, y) in enumerate(puntos_marcados[:15]):
            nueva_fila[f'x_punto {i + 1}'] = x
            nueva_fila[f'y_punto {i + 1}'] = y

        # Añadir la nueva fila al dataset
        df = df.append(nueva_fila, ignore_index=True)

        # Guardar el dataset actualizado
        df.to_csv(ruta_dataset, index=False)
        print("Dataset actualizado y guardado exitosamente.")
    except Exception as e:
        print(f"Error al actualizar el dataset: {e}")

## Procesar Imagen

In [ ]:
def marcar_puntos(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        puntos_marcados.append((x, y))
        cv2.circle(param, (x, y), 5, (255, 0, 0), -1)  # Dibuja el punto en la imagen
        cv2.imshow('Imagen con Grilla', param)

# Función para mostrar la imagen y permitir marcar puntos con el mouse
def mostrar_imagen_con_mouse(imagen):
    cv2.imshow('Imagen con Grilla', imagen)
    cv2.setMouseCallback('Imagen con Grilla', marcar_puntos, imagen)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Función para procesar una imagen individual y marcar puntos
def procesar_imagen_individual(ruta_imagen, punto_referencia, pixeles_por_3mm, directorio_salida):
    global puntos_marcados
    puntos_marcados = []  # Reiniciar la lista de puntos marcados para cada imagen

    imagen = cv2.imread(ruta_imagen)
    if imagen is None:
        print(f"No se pudo cargar la imagen: {ruta_imagen}")
        return None, []

    pixeles_por_mm = pixeles_por_3mm
    num_columnas, num_filas = 8, 5

    dibujar_puntos(imagen, punto_referencia, punto_referencia)  # Dibujar el punto de referencia

    centro_x, centro_y, tamanio_celda = calcular_centro_grilla(punto_referencia[0], punto_referencia[1], pixeles_por_mm, num_filas)

    matriz_grilla = crear_matriz(num_columnas, num_filas)  # Inicializar la matriz sin puntos marcados

    # Dibujar la grilla en la imagen antes de mostrarla para marcar puntos
    dibujar_grilla(imagen, centro_x, centro_y, tamanio_celda, num_columnas, num_filas, matriz_grilla)

    # Mostrar la imagen con la grilla y permitir marcar puntos con el mouse
    mostrar_imagen_con_mouse(imagen)

    for punto_marcado in puntos_marcados:
        dibujar_puntos(imagen, punto_referencia, punto_marcado)
        grilla_x, grilla_y = obtener_posicion_grilla(punto_marcado[0], punto_marcado[1], centro_x, centro_y, tamanio_celda, num_columnas, num_filas)

        # Verificar que los índices están dentro de los límites
        if 0 <= grilla_x < num_columnas and 0 <= grilla_y < num_filas:
            matriz_grilla[grilla_y][grilla_x] = 1  # Marcar el punto en la matriz
        else:
            print(f"Punto fuera de la grilla: {punto_marcado} -> ({grilla_x}, {grilla_y})")

    # Asegurarse de que directorio_salida es un directorio
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)

    ruta_salida = os.path.join(directorio_salida, os.path.basename(ruta_imagen))
    cv2.imwrite(ruta_salida, imagen)
    
    return matriz_grilla, puntos_marcados

In [ ]:
def procesar_conjunto_datos(ruta_conjunto_datos, directorio_salida, ruta_salida_matriz):
    conjunto_datos = pd.read_csv(ruta_conjunto_datos)
    os.makedirs(directorio_salida, exist_ok=True)

    matrices = []
    puntos_totales = []

    for indice, fila in conjunto_datos.iterrows():
        nombre_imagen = fila["imagen"]
        ruta_imagen = os.path.join('..', 'imagenes', 'seleccionadas', 'inicio', nombre_imagen)

        punto_referencia = (int(fila['punto_referencia_x']), int(fila['punto_referencia_y']))
        mm_por_pixel = fila['mm_por_pixel']

        matriz_grilla, puntos_marcados = procesar_imagen_individual(ruta_imagen, punto_referencia, mm_por_pixel, directorio_salida)
        if matriz_grilla is not None:
            matrices.append({
                'imagen': nombre_imagen,
                'matriz': matriz_grilla
            })

            puntos_imagen = {'imagen': nombre_imagen}
            for i, punto in enumerate(puntos_marcados, start=1):
                puntos_imagen[f'coordenada_x_punto {i}'] = punto[0]
                puntos_imagen[f'coordenada_y_punto {i}'] = punto[1]
            puntos_totales.append(puntos_imagen)

    # Guardar las matrices en un archivo CSV
    df_matrices = pd.DataFrame(matrices)
    df_matrices.to_csv(ruta_salida_matriz, index=False)

    # Guardar los puntos marcados en otro archivo CSV
    df_puntos = pd.DataFrame(puntos_totales)
    ruta_salida_puntos = ruta_salida_matriz.replace('.csv', '_puntos.csv')
    df_puntos.to_csv(ruta_salida_puntos, index=False)

    print("\nProcesamiento completado.")
    print(f"Directorio de salida: {os.path.abspath(directorio_salida)}")
    print(f"Matriz guardada en: {os.path.abspath(ruta_salida_matriz)}")
    print(f"Puntos guardados en: {os.path.abspath(ruta_salida_puntos)}")

In [ ]:
ruta_conjunto_datos = '../datos/eliminar/df.csv'
directorio_salida = '../imagenes/procesadas/'
ruta_salida_matriz = '../datos/matriz_grilla_final.csv'
procesar_conjunto_datos(ruta_conjunto_datos, directorio_salida, ruta_salida_matriz)